In [1]:
!pip install transformers

     |████████████████████████████████| 2.1 MB 6.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 39.3 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 54.8 MB/s eta 0:00:01


In [13]:
!pip install spacy

In [14]:
!pip install scispacy

In [2]:
!pip install scispacy==0.4.0
!pip install spacy==3.0.6
!pip install spacy-alignments==0.8.3
!pip install spacy-legacy==3.0.5
!pip install spacy-transformers==1.0.2

     |████████████████████████████████| 44 kB 505 kB/s eta 0:00:011
     |████████████████████████████████| 13.5 MB 6.4 MB/s eta 0:00:01
     |████████████████████████████████| 71 kB 2.4 MB/s  eta 0:00:01
     |████████████████████████████████| 188 kB 45.0 MB/s eta 0:00:01
     |████████████████████████████████| 12.9 MB 6.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 40.8 MB/s eta 0:00:01
     |████████████████████████████████| 456 kB 48.8 MB/s eta 0:00:01
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.1
    Uninstalling catalogue-2.0.1:
      Successfully uninstalled catalogue-2.0.1
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.0
    Uninstalling srsly-2.4.0:
      Successfully uninstalled srsly-2.4.0
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.1
    Uninstalling thinc-8.0.1:
      Successfully uninstalled thinc-8.0.1
  Attempting uninstall: spacy-legacy
    Found existing ins

In [3]:
import os
import sys
# science
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
# process
import joblib
from tqdm import tqdm
from functools import partial
# ml
import torch
from transformers import AutoModel, AutoTokenizer

In [4]:
torch.cuda.is_available()

True

In [5]:
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

In [6]:
import pandas as pd
df = pd.read_csv('final_data.csv')
df.head(3)

,ICD Block Names,Title,Research Summary,Inclusion Criteria
0,Abnormal findings on diagnostic imaging and in...,Checkpoint inhibitor-induced liver injury (ChI...,â¢ Immune checkpoint inhibitors are proven ca...,Both patient groups and control group: Aged 1...
1,"Abnormal findings on examination of blood, wit...",Exploring the patient experience of a diagnosi...,Research Question: Exploring the patient exper...,â Coded diagnosis of pre-diabetes and have b...
2,"Abnormal findings on examination of blood, wit...",Assessment of the Impact of a Personalised Nut...,This study will determine if DNA-based dietary...,In order to be eligible to participate in this...


In [7]:
!python -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_scibert-0.4.0.tar.gz

     |████████████████████████████████| 423.3 MB 6.1 kB/s  eta 0:00:01    |█████████▏                      | 120.7 MB 6.1 MB/s eta 0:00:50     |█████████▌                      | 125.9 MB 72.2 MB/s eta 0:00:05     |██████████                      | 132.5 MB 72.2 MB/s eta 0:00:05     |███████████                     | 145.5 MB 72.2 MB/s eta 0:00:04     |███████████████████▍            | 256.6 MB 31.1 MB/s eta 0:00:06
  Created wheel for en-core-sci-scibert: filename=en_core_sci_scibert-0.4.0-py3-none-any.whl size=423355915 sha256=ea471f8ca6179d112becaea0125aa463164b4b94fcee5992ae19fd11490ba543
  Stored in directory: /home/ec2-user/.cache/pip/wheels/00/bd/d0/13d731537d2cf1e745dbac1b3edd1e0254aea5318dd662b19c
Successfully built en-core-sci-scibert


In [16]:
'''
Download and unzip data for finetuning
'''

!pip install gdown
!gdown https://drive.google.com/uc?id=1OletxmPYNkz2ltOr9pyT0b0iBtUWxslh
!unzip -o NERdata.zip

Archive:  NERdata.zip
  inflating: BC2GM/devel.tsv         
  inflating: BC2GM/test.tsv          
  inflating: BC2GM/train.tsv         
  inflating: BC2GM/train_dev.tsv     
  inflating: BC4CHEMD/devel.tsv      
  inflating: BC4CHEMD/test.tsv       
  inflating: BC4CHEMD/train.tsv      
  inflating: BC4CHEMD/train_dev.tsv  
   creating: BC5CDR-chem/
  inflating: BC5CDR-chem/devel.tsv   
  inflating: BC5CDR-chem/test.tsv    
  inflating: BC5CDR-chem/train.tsv   
  inflating: BC5CDR-chem/train_dev.tsv  
   creating: BC5CDR-disease/
  inflating: BC5CDR-disease/devel.tsv  
  inflating: BC5CDR-disease/test.tsv  
  inflating: BC5CDR-disease/train.tsv  
  inflating: BC5CDR-disease/train_dev.tsv  
   creating: JNLPBA/
  inflating: JNLPBA/devel.tsv        
  inflating: JNLPBA/test.tsv         
  inflating: JNLPBA/train.tsv        
  inflating: JNLPBA/train_dev.tsv    
   creating: linnaeus/
  inflating: linnaeus/devel.tsv      
  inflating: linnaeus/test.tsv       
  inflating: linnaeus/train.t

In [8]:
#Import all required libraries
import spacy
import random
import time
import numpy as np
from spacy.util import minibatch, compounding
import sys
from spacy import displacy
from itertools import chain
import matplotlib.pyplot as plt 
from matplotlib.ticker import MaxNLocator

In [18]:
def load_data_spacy(file_path):
    ''' Converts data from:
    word \t label \n word \t label \n \n word \t label
    to: sentence, {entities : [(start, end, label), (stard, end, label)]}
    '''
    file = open(file_path, 'r')
    training_data, entities, sentence, unique_labels = [], [], [], []
    current_annotation = None
    start =0
    end = 0 # initialize counter to keep track of start and end characters
    for line in file:
        line = line.strip("\n").split("\t")
        # lines with len > 1 are words
        if len(line) > 1:
            label = line[1]
            if(label != 'O'):
                label = line[1]+"_Disease"     # the .txt is formatted: label \t word, label[0:2] = label_type
            #label_type = line[0][0] # beginning of annotations - "B", intermediate - "I"
            word = line[0]
            sentence.append(word)
            start = end
            end += (len(word) + 1)  # length of the word + trailing space
           
            if label == 'I_Disease' :  # if at the end of an annotation
                entities.append(( start,end-1, label))  # append the annotation
                              
            if label == 'B_Disease':                         # if beginning new annotation
                entities.append(( start,end-1, label))# start annotation at beginning of word
                
           
           
            if label != 'O' and label not in unique_labels:
                unique_labels.append(label)
 
        # lines with len == 1 are breaks between sentences
        if len(line) == 1:
            if(len(entities) > 0):
                sentence = " ".join(sentence)
                training_data.append([sentence, {'entities' : entities}])
            # reset the counters and temporary lists
            end = 0 
            start = 0
            entities, sentence = [], []
            
    file.close()
    return training_data, unique_labels

In [21]:
directories = ['./BC5CDR-chem','./NCBI-disease','./BC5CDR-disease']

In [22]:
FULL_TRAIN_DATA = []
FULL_VALID_DATA = []
FULL_TEST_DATA = []

FULL_TRAIN_LABELS = []
FULL_VALID_LABELS = []
FULL_TEST_LABELS = []

for dic in directories:
    train_data, train_labels = load_data_spacy("{}/train.tsv".format(dic))
    valid_data, valid_labels = load_data_spacy("{}/train_dev.tsv".format(dic))
    test_data, test_labels = load_data_spacy("{}/test.tsv".format(dic))
    
    FULL_TRAIN_DATA.extend(train_data) , FULL_TRAIN_LABELS.extend(train_labels)
    FULL_VALID_DATA.extend(valid_data) , FULL_VALID_LABELS.extend(valid_labels)
    FULL_TEST_DATA.extend(test_data) , FULL_TEST_LABELS.extend(test_labels)

In [23]:
import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

"""
Convert data from spacy 2 format to spacy 3
"""

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object
def conv_data(data,name):
    for text, annot in tqdm(data): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        db.add(doc)

    db.to_disk("./{}.spacy".format(str(name)))

In [24]:
FULL_TRAIN_DATA = FULL_TRAIN_DATA+FULL_VALID_DATA """Concat Train and Validation for more training data"""

In [25]:
conv_data(FULL_TRAIN_DATA,"train")
#conv_data(FULL_VALID_DATA,"valid")
conv_data(FULL_TEST_DATA,"test")

100%|██████████| 6472/6472 [00:04<00:00, 1572.16it/s]


In [10]:
!pip install murmurhash

In [11]:
!python -m spacy init fill-config ./base_config_scibert.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [5]:
#!mkdir output

In [26]:
!python -m spacy train ./config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./test.spacy --gpu-id 0

ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-05-07 07:53:35,757] [INFO] Set up nlp object from config
[2021-05-07 07:53:35,772] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-05-07 07:53:35,776] [INFO] Created vocabulary
[2021-05-07 07:53:40,820] [WARNING] [W112] The model specified to use for initial vectors (en_core_sci_scibert) has no vectors. This is almost certainly a mistake.
[2021-05-07 07:53:40,827] [INFO] Added vectors: en_core_sci_scibert
[2021-05-07 07:53:40,827] [INFO] Finished initializing nlp object
[2021-05-07 07:54:08,851] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     35.30    0.00    0.

In [47]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [16]:
!pip install -U spacy[cuda101]

     |████████████████████████████████| 121.7 MB 64.2 MB/s eta 0:00:01


In [52]:
#!pip install -U spacy[cuda]

  Using cached cupy-8.6.0.tar.gz (1.3 MB)
    ERROR: Command errored out with exit status 1:
     command: /home/ec2-user/anaconda3/envs/pytorch_p36/bin/python3.6 -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-9hlacraa/cupy_13bb3d9dd4784812952b46ef16414a4d/setup.py'"'"'; __file__='"'"'/tmp/pip-install-9hlacraa/cupy_13bb3d9dd4784812952b46ef16414a4d/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-v6mmbuf8
         cwd: /tmp/pip-install-9hlacraa/cupy_13bb3d9dd4784812952b46ef16414a4d/
    Complete output (50 lines):
    Options: {'package_name': 'cupy', 'long_description': None, 'wheel_libs': [], 'wheel_includes': [], 'wheel_metadata': None, 'no_rpath': False, 'profile': False, 'linetrace': False, 'annotate': False, 'no_cuda': False, 'use_hip': False}
    
    -------- Configuring Module: cuda ---

In [33]:
nlp1 = spacy.load(R"./output/model-best") #load the best model
doc = nlp1(df['Research Summary'][5]) # input sample text

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter